In [ ]:
#  from google.colab import drive
# # drive.mount('/content/drive/')

ValueError: mount failed

In [ ]:
# !unzip /content/drive/MyDrive/datasets/dataset-pcb.zip -d /content/

unzip:  cannot find or open /content/drive/MyDrive/datasets/dataset-pcb.zip, /content/drive/MyDrive/datasets/dataset-pcb.zip.zip or /content/drive/MyDrive/datasets/dataset-pcb.zip.ZIP.


In [1]:
!nvidia-smi


Mon Feb  2 09:46:06 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
os.getcwd()

'/content'

In [3]:
!git clone https://github.com/ultralytics/yolov5.git #this is the yolov5 directory

Cloning into 'yolov5'...
remote: Enumerating objects: 17783, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 17783 (delta 2), reused 0 (delta 0), pack-reused 17776 (from 2)
Receiving objects: 100% (17783/17783), 16.94 MiB | 14.00 MiB/s, done.
Resolving deltas: 100% (12130/12130), done.


In [4]:
%cd yolov5

/content/yolov5


In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0


In [9]:
!pwd


/content/yolov5


In [13]:
!ls /content


sample_data  yolov5


In [14]:
!ls /content
!cat /content/dataset.yaml


sample_data  yolov5
cat: /content/dataset.yaml: No such file or directory


In [17]:
!cat ../dataset.yaml


cat: ../dataset.yaml: No such file or directory


In [13]:
!python train.py --img 416 --batch 16 --epochs 300 \
--data ../dataset.yaml \
--weights yolov5s.pt \
--cache \
--name pcb_1st


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2026-02-01 18:30:21.702731: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769970621.722999    2272 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769970621.729044    2272 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769970621.744026    2272 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769970621.744056    2272 computation_placer.cc:177] computation placer already registere

In [ ]:
# Save this as a reference for your backend API
api_example = '''
# Flask API Example
from flask import Flask, request, jsonify
import torch
from PIL import Image
import io

app = Flask(__name__)

# Load model once at startup
model = torch.hub.load('ultralytics/yolov5', 'custom', 
                       path='pcb_defect_model.pt')
model.conf = 0.25

@app.route('/detect', methods=['POST'])
def detect_defects():
    if 'image' not in request.files:
        return jsonify({'error': 'No image provided'}), 400
    
    # Read image
    image_file = request.files['image']
    image = Image.open(io.BytesIO(image_file.read()))
    
    # Run detection
    results = model(image)
    
    # Parse results
    detections = []
    for *box, conf, cls in results.xyxy[0].tolist():
        detections.append({
            'class': model.names[int(cls)],
            'confidence': float(conf),
            'bbox': [float(x) for x in box]
        })
    
    return jsonify({
        'success': True,
        'detections': detections,
        'count': len(detections)
    })

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
'''

print(api_example)

## API Integration Example

Here's how to integrate the model in a Flask/FastAPI backend:

In [ ]:
# Example: Load and use the model for inference
import torch

# Load the trained model
model_path = '/content/production_models/pcb_defect_model.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

# Configure model
model.conf = 0.25  # Confidence threshold
model.iou = 0.45   # NMS IoU threshold

print("Model loaded successfully!")
print(f"Classes: {model.names}")

# Example inference function for your application
def detect_pcb_defects(image_path):
    """
    Detect defects in a PCB image
    
    Args:
        image_path: Path to the PCB image
    
    Returns:
        results: Detection results with bounding boxes and labels
    """
    results = model(image_path)
    return results

# Test the function
# results = detect_pcb_defects('../test_images/test_1.jpg')
# results.show()  # Display results
# results.save()  # Save results

# Using the Model in Your Application

Example Python code to load and use the trained model:

In [ ]:
# List all available models for production
print("Available models for production:")
print("-" * 50)
for file in os.listdir(models_dir):
    file_path = os.path.join(models_dir, file)
    size_mb = os.path.getsize(file_path) / (1024 * 1024)
    print(f"📦 {file} ({size_mb:.2f} MB)")
print("-" * 50)

In [ ]:
# Optional: Export to TorchScript (for production deployment)
!python export.py --weights runs/train/pcb_1st/weights/best.pt \
--include torchscript \
--img 416

# Copy TorchScript model
ts_model = 'runs/train/pcb_1st/weights/best.torchscript'
if os.path.exists(ts_model):
    shutil.copy(ts_model, os.path.join(models_dir, 'pcb_defect_model.torchscript'))
    print(f"✓ TorchScript model saved to: {models_dir}/pcb_defect_model.torchscript")

In [ ]:
# Export model to ONNX format (for web/mobile deployment)
!python export.py --weights runs/train/pcb_1st/weights/best.pt \
--include onnx \
--img 416 \
--device cpu

# Copy ONNX model to production directory
onnx_model = 'runs/train/pcb_1st/weights/best.onnx'
if os.path.exists(onnx_model):
    shutil.copy(onnx_model, os.path.join(models_dir, 'pcb_defect_model.onnx'))
    print(f"✓ ONNX model saved to: {models_dir}/pcb_defect_model.onnx")

In [ ]:
# Create a models directory for production use
import os
import shutil

# Create models directory in project root
models_dir = '/content/production_models'
os.makedirs(models_dir, exist_ok=True)

# Copy the best trained model
best_model_path = 'runs/train/pcb_1st/weights/best.pt'
if os.path.exists(best_model_path):
    shutil.copy(best_model_path, os.path.join(models_dir, 'pcb_defect_model.pt'))
    print(f"✓ Model saved to: {models_dir}/pcb_defect_model.pt")
else:
    print("Model not found. Make sure training completed successfully.")

# Save and Export Model for Production

After training, the model is automatically saved. We'll copy it to a standard location and export it in multiple formats for use in full stack applications.

In [57]:
!zip -r /content/drive/MyDrive/datasets/runs-pcb.zip /content/yolov5/runs

	zip warning: name not matched: /content/yolov5/runs

zip error: Nothing to do! (try: zip -r /content/drive/MyDrive/datasets/runs-pcb.zip . -i /content/yolov5/runs)


In [ ]:
!python val.py --weights runs/train/pcb_1st/weights/best.pt --data dataset.yaml 

In [ ]:
!python detect.py --source ../test_images/test_1.jpg  --weights runs/train/pcb_1st/weights/best.pt